### Deep learning-based human activity recognition using CNN, ConvLSTM, and LRCN

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/human-activity-recognition/data')

In [ ]:
!unrar x UCF50.rar

Streaming output truncated to the last 5000 lines.
Extracting  UCF50/HorseRace/v_HorseRace_g16_c03.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g16_c04.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g16_c05.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c01.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c02.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c03.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c04.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g17_c05.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g18_c01.avi                       23%  OK 
Extracting  UCF50/HorseRace/v_HorseRace_g18_c02.avi                       23%  OK 
Extracting  UCF50/HorseRace/v

In [ ]:
%cd ..

/content/drive/MyDrive/human-activity-recognition/data


In [ ]:
%cd HMDB51

/content/drive/MyDrive/human-activity-recognition/data/HMDB51/HMDB51/HMDB51


In [ ]:
!ls

hmdb51.zip


In [ ]:
import os
os.chdir('/content/drive/MyDrive/human-activity-recognition/data/HMDB51')

In [ ]:
!unzip hmdb51.zip

Streaming output truncated to the last 5000 lines.
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10005.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10006.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10007.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10008.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10009.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10010.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10011.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10012.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10013.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10014.jpg  
  inflating: HMDB51/walk/TheLittleShopofHorrors_walk_f_cm_np1_fr_med_12/10015.jpg  
  inflating: HMDB51/walk/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd HMDB51

/content/drive/MyDrive/human-activity-recognition/data/HMDB51/HMDB51/HMDB51


In [ ]:
!ls

brush_hair    eat	  kiss	      shake_hands     sword
cartwheel     fall_floor  laugh       shoot_ball      sword_exercise
catch	      fencing	  pick	      shoot_bow       talk
chew	      flic_flac   pour	      shoot_gun       throw
clap	      golf	  pullup      sit	      turn
climb	      handstand   punch       situp	      walk
climb_stairs  hit	  push	      smile	      wave
dive	      hug	  pushup      smoke
draw_sword    jump	  ride_bike   somersault
dribble       kick	  ride_horse  stand
drink	      kick_ball   run	      swing_baseball


### Implementation

In [ ]:
%cd ..

/content/drive/MyDrive/human-activity-recognition/data


In [3]:
os.chdir('/content/drive/MyDrive/human-activity-recognition/data')

In [4]:
# !pip install -q torch torchvision opencv-python-headless sklearn tqdm

import os, random, time, math, shutil
from typing import List, Tuple
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision import transforms

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# By default expect datasets to live in Drive. Edit these paths as needed.
# The notebook will run two separate experiments (paper style): UCF50 and HMDB51.
UCF50_ROOT = "/content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50"
HMDB51_ROOT = "/content/drive/MyDrive/human-activity-recognition/data/HMDB51/HMDB51/HMDB51"

SEED = 42
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed()

def list_top_files(root, n=20):
    for i, (r,d,f) in enumerate(os.walk(root)):
        print(r)
        for j, file in enumerate(f[:10]):
            print("   ", file)
        if i>n: break

Device: cuda


In [5]:
# [dataset collector]
# Directories:
#  - UCF50-like: root/.../<class>/*.avi (video files)
#  - HMDB51-like frames: root/.../<class>/<video_folder>/*.jpg  (each video_folder is a sequence)

VIDEO_EXTS = (".mp4", ".avi", ".mkv", ".mov")
FRAME_EXTS = (".jpg", ".jpeg", ".png")

def collect_dataset_items(root_dir: str):
    """
    Walk root_dir. For each class directory (folder containing either video files or folders-of-frames),
    return list of items where item is:
      ("video", path_to_video, class_name)
      ("frames", [sorted_frame_paths], class_name)
    """
    items = []
    classes = []
    if not os.path.exists(root_dir):
        print("Warning: dataset root not found:", root_dir)
        return items, {}
    for class_name in sorted(os.listdir(root_dir)):
        class_path = os.path.join(root_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        classes.append(class_name)
        # check if this class_path directly contains video files
        files = sorted(os.listdir(class_path))
        has_video_files = any(f.lower().endswith(VIDEO_EXTS) for f in files)
        if has_video_files:
            for f in files:
                if f.lower().endswith(VIDEO_EXTS):
                    items.append(("video", os.path.join(class_path, f), class_name))
        else:
            # treat each subfolder as a video (frames)
            for sub in sorted(os.listdir(class_path)):
                vf = os.path.join(class_path, sub)
                if not os.path.isdir(vf):
                    continue
                frames = sorted([os.path.join(vf, x) for x in os.listdir(vf)
                                 if x.lower().endswith(FRAME_EXTS)])
                if len(frames) >= 1:
                    items.append(("frames", frames, class_name))
    label_map = {c:i for i,c in enumerate(sorted(classes))}
    return items, label_map

print("UCF50 listing sample:")
list_top_files(UCF50_ROOT, n=3)
print("HMDB51 listing sample:")
list_top_files(HMDB51_ROOT, n=3)

UCF50 listing sample:
/content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50
/content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50/MilitaryParade
    v_MilitaryParade_g01_c01.avi
    v_MilitaryParade_g01_c02.avi
    v_MilitaryParade_g01_c03.avi
    v_MilitaryParade_g01_c04.avi
    v_MilitaryParade_g01_c05.avi
    v_MilitaryParade_g01_c06.avi
    v_MilitaryParade_g01_c07.avi
    v_MilitaryParade_g02_c01.avi
    v_MilitaryParade_g02_c02.avi
    v_MilitaryParade_g02_c03.avi
/content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50/Mixing
    v_Mixing_g01_c01.avi
    v_Mixing_g01_c02.avi
    v_Mixing_g01_c03.avi
    v_Mixing_g01_c04.avi
    v_Mixing_g01_c05.avi
    v_Mixing_g01_c06.avi
    v_Mixing_g01_c07.avi
    v_Mixing_g01_c08.avi
    v_Mixing_g02_c01.avi
    v_Mixing_g02_c02.avi
/content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50/Nunchucks
    v_Nunchucks_g01_c01.avi
    v_Nunchucks_g01_c02.avi
    v_Nunchucks_g01_c03.avi
    v_Nunchuck

In [6]:
# CELL 4: Dataset class
def sample_frames_from_video_file(video_path: str, num_frames: int, resize=(224,224)):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total <= 0:
        cap.release()
        return None
    idxs = np.linspace(0, max(0, total-1), num_frames).astype(int)
    frames = []
    idx_set = set(idxs.tolist())
    idx = 0
    got = 0
    while True:
        ret, frame = cap.read()
        if not ret: break
        if idx in idx_set:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, resize)
            frames.append(frame)
            got += 1
            if got >= num_frames: break
        idx += 1
    cap.release()
    if len(frames)==0: return None
    while len(frames) < num_frames:
        frames.append(frames[-1].copy())
    return np.stack(frames).astype(np.float32)/255.0

def sample_frames_from_frame_list(frame_list: List[str], num_frames: int, resize=(224,224)):
    # frame_list is a sorted list of frame filepaths
    total = len(frame_list)
    if total == 0: return None
    idxs = np.linspace(0, max(0, total-1), num_frames).astype(int)
    frames=[]
    for i in idxs:
        img = cv2.imread(frame_list[int(i)])
        if img is None:
            # missing file -then skip or fail
            img = np.zeros((resize[1], resize[0], 3), dtype=np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, resize)
        frames.append(img)
    return np.stack(frames).astype(np.float32)/255.0

class MixedVideoDataset(Dataset):
    """
    items: list of ("video", path, class) or ("frames", [paths], class)
    label_map: class->idx
    """
    def __init__(self, items, label_map, num_frames=16, resize=(224,224), transform=None):
        self.items = items
        self.label_map = label_map
        self.num_frames = num_frames
        self.resize = resize
        self.transform = transform

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        typ, entry, cls = self.items[idx]
        if typ == "video":
            frames = sample_frames_from_video_file(entry, self.num_frames, resize=self.resize)
        else:
            frames = sample_frames_from_frame_list(entry, self.num_frames, resize=self.resize)
        if frames is None:
            raise RuntimeError(f"Could not load frames for {entry}")
        # frames: (T,H,W,3)
        if self.transform:
            fps = []
            for f in frames:
                fps.append(self.transform(f))
            frames_t = torch.stack(fps)  # (T,C,H,W)
        else:
            frames_t = torch.tensor(frames).permute(0,3,1,2)
        label_idx = self.label_map[cls]
        return frames_t, label_idx

In [7]:
# CELL 5: Models (CNN baseline, LRCN, ConvLSTM) + attention
class TemporalAttention(nn.Module):
    def __init__(self, feat_dim:int):
        super().__init__()
        self.w = nn.Linear(feat_dim, 1)
    def forward(self, x):  # x: (B,T,D)
        scores = self.w(x)  # (B,T,1)
        w = F.softmax(scores, dim=1)
        agg = (x * w).sum(dim=1)
        return agg, w

class FrameCNN(nn.Module):
    def __init__(self, n_classes:int, pretrained=True):
        super().__init__()
        res = models.resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(res.children())[:-1])  # (N,feat,1,1)
        feat_dim = res.fc.in_features
        self.fc = nn.Linear(feat_dim, n_classes)
    def forward(self, frames):  # frames: (B,T,C,H,W)
        B,T,C,H,W = frames.shape
        frames = frames.view(B*T, C, H, W)
        feats = self.backbone(frames).view(B, T, -1)  # (B,T,feat)
        pooled = feats.mean(dim=1)
        logits = self.fc(pooled)
        return logits, feats

class LRCN(nn.Module):
    def __init__(self, n_classes:int, hidden_size=512, pretrained=True, use_att=False, bidir=False):
        super().__init__()
        res = models.resnet34(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(res.children())[:-1])
        feat_dim = res.fc.in_features
        self.lstm = nn.LSTM(input_size=feat_dim, hidden_size=hidden_size, batch_first=True, bidirectional=bidir)
        final_dim = hidden_size * (2 if bidir else 1)
        self.use_att = use_att
        if use_att:
            self.att = TemporalAttention(final_dim)
        self.classifier = nn.Linear(final_dim, n_classes)
    def forward(self, frames):
        B,T,C,H,W = frames.shape
        x = frames.view(B*T, C, H, W)
        feats = self.backbone(x).view(B, T, -1)
        out, _ = self.lstm(feats)
        if self.use_att:
            agg, weights = self.att(out)
            logits = self.classifier(agg)
            return logits, weights
        else:
            logits = self.classifier(out[:, -1, :])
            return logits, None
# --- REPLACEMENT FOR YOUR ConvLSTMCell + ConvLSTM IN CELL 5 ---

class ConvLSTMCell(nn.Module):
    def __init__(self, in_ch, hidden_ch, kernel_size=3):
        super().__init__()
        padding = kernel_size // 2
        self.hidden_ch = hidden_ch

        self.conv = nn.Conv2d(in_ch + hidden_ch, 4 * hidden_ch,
                              kernel_size, padding=padding, bias=False)
        self.bn = nn.BatchNorm2d(4 * hidden_ch)

    def forward(self, x, h, c):
        # x, h, c shapes: (B, C, H, W)
        combined = torch.cat([x, h], dim=1)
        gates = self.bn(self.conv(combined))

        i, f, o, g = torch.chunk(gates, 4, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_new = f * c + i * g
        h_new = o * torch.tanh(c_new)

        return h_new, c_new


class ConvLSTM(nn.Module):
    def __init__(self, n_classes=50, hidden_channels=128):
        super().__init__()

        # Use a pretrained CNN as encoder (MUCH better)
        base = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        layers = list(base.children())[:-3]          # output: (B,256,H/8,W/8)
        self.encoder = nn.Sequential(*layers)

        self.enc_out_ch = 256  # from resnet18
        self.hidden_ch = hidden_channels

        self.cell = ConvLSTMCell(self.enc_out_ch, hidden_channels)

        self.dropout = nn.Dropout(0.3)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(hidden_channels, n_classes)

    def forward(self, frames):
        # frames: (B,T,C,H,W)
        B, T, C, H, W = frames.shape

        # compute encoder spatial dims
        with torch.no_grad():
            dummy = self.encoder(frames[:, 0])
        _, Cenc, Henc, Wenc = dummy.shape

        # initialize hidden & cell state
        h = torch.zeros(B, self.hidden_ch, Henc, Wenc, device=frames.device)
        c = torch.zeros_like(h)

        for t in range(T):
            x_enc = self.encoder(frames[:, t])  # (B,256,H/8,W/8)
            h, c = self.cell(x_enc, h, c)

        out = self.pool(h).view(B, -1)
        out = self.dropout(out)
        logits = self.fc(out)

        return logits

In [8]:
# CELL 6 - Training & evaluation helpers
def collate_fn(batch):
    frames = torch.stack([b[0] for b in batch])
    labels = torch.tensor([b[1] for b in batch], dtype=torch.long)
    return frames, labels

def train_one_epoch(model, loader, optimizer, device, scaler=None):
    model.train()
    total_loss=0.0
    n=0
    for frames, labels in tqdm(loader, desc="Train", leave=False):
        frames = frames.to(device); labels = labels.to(device)
        optimizer.zero_grad()
        if scaler:
            with torch.cuda.amp.autocast():
                out = model(frames)
                logits = out[0] if isinstance(out, (tuple,list)) else out
                loss = F.cross_entropy(logits, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            out = model(frames)
            logits = out[0] if isinstance(out, (tuple,list)) else out
            loss = F.cross_entropy(logits, labels)
            loss.backward()
            optimizer.step()
        total_loss += loss.item() * frames.size(0)
        n += frames.size(0)
    return total_loss / max(1,n)

@torch.no_grad()
def evaluate(model, loader, device, label_names):
    model.eval()
    preds=[]; trues=[]
    for frames, labels in tqdm(loader, desc="Eval", leave=False):
        frames = frames.to(device)
        out = model(frames)
        logits = out[0] if isinstance(out, (tuple,list)) else out
        probs = F.softmax(logits, dim=1).cpu().numpy()
        y_pred = probs.argmax(axis=1).tolist()
        preds += y_pred
        trues += labels.tolist()
    acc = accuracy_score(trues, preds)
    report = classification_report(trues, preds, target_names=label_names, zero_division=0)
    cm = confusion_matrix(trues, preds)
    return acc, report, cm

In [9]:
# CELL 7: Grad-CAM helper (frame-level visualization)
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hooks = []
        self._register_hooks()
    def _register_hooks(self):
        def fwd(m, i, o):
            self.activations = o.detach()
        def bwd(m, gi, go):
            self.gradients = go[0].detach()
        self.hooks.append(self.target_layer.register_forward_hook(fwd))
        self.hooks.append(self.target_layer.register_backward_hook(bwd))
    def __call__(self, img_tensor, class_idx=None):
        # img_tensor: (C,H,W) CPU tensor (not batched)
        self.model.zero_grad()
        x = img_tensor.to(next(self.model.parameters()).device)
        out = self.model(x.unsqueeze(0))
        logits = out[0] if isinstance(out, (tuple,list)) else out
        if class_idx is None:
            class_idx = logits.argmax(dim=1).item()
        loss = logits[0, class_idx]
        loss.backward(retain_graph=True)
        grads = self.gradients[0]  # (C,H,W)
        acts = self.activations[0]
        weights = grads.mean(dim=(1,2))
        cam = (weights[:,None,None] * acts).sum(dim=0).cpu().numpy()
        cam = np.maximum(cam,0)
        cam = (cam - cam.min())/(cam.max()-cam.min()+1e-8)
        return cam
    def close(self):
        for h in self.hooks: h.remove()

In [10]:
# CELL 8
# (UCF50 run, HMDB51 run). Each run trains CNN, LRCN, ConvLSTM.
def run_experiment(root_dir, experiment_name, num_frames=16, image_size=(224,224),
                   batch_size=8, num_epochs=30, lr=1e-4, use_amp=True, pretrained=True):
    print(f"=== Experiment: {experiment_name} on root {root_dir} ===")
    items, label_map = collect_dataset_items(root_dir)
    if len(items)==0:
        print("No items found under", root_dir)
        return
    # Build label_names ordered by index
    label_names = [None] * len(label_map)
    for k,v in label_map.items():
        label_names[v]=k

    # train/val/test split at video-level
    random.Random(SEED).shuffle(items)
    n = len(items)
    n_test = max(1, int(n*0.15))
    n_val = max(1, int(n*0.15))
    test_items = items[:n_test]
    val_items = items[n_test:n_test+n_val]
    train_items = items[n_test+n_val:]
    print("Counts (train/val/test):", len(train_items), len(val_items), len(test_items))

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])

    train_ds = MixedVideoDataset(train_items, label_map, num_frames=num_frames, resize=image_size, transform=transform)
    val_ds = MixedVideoDataset(val_items, label_map, num_frames=num_frames, resize=image_size, transform=transform)
    test_ds = MixedVideoDataset(test_items, label_map, num_frames=num_frames, resize=image_size, transform=transform)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1)

    # Instantiate models
    n_classes = len(label_map)
    print("n_classes:", n_classes)

    # 1) CNN baseline (frame-averaging)
    cnn = FrameCNN(n_classes=n_classes, pretrained=pretrained).to(DEVICE)
    opt_cnn = optim.Adam(cnn.parameters(), lr=lr)
    scaler = torch.cuda.amp.GradScaler() if (use_amp and DEVICE.type=='cuda') else None

    best_val = 0.0
    best_model_path = f"{experiment_name}_cnn_best.pth"
    print("Training CNN baseline...")
    for epoch in range(1, num_epochs+1):
        t0=time.time()
        loss = train_one_epoch(cnn, train_loader, opt_cnn, DEVICE, scaler=scaler)
        val_acc, _, _ = evaluate(cnn, val_loader, DEVICE, label_names)
        print(f"[CNN] Epoch {epoch}/{num_epochs} loss {loss:.4f} val_acc {val_acc:.4f} time {time.time()-t0:.1f}s")
        if val_acc > best_val:
            best_val = val_acc
            torch.save(cnn.state_dict(), best_model_path)
    print("CNN best val:", best_val)

    # 2) LRCN (CNN backbone + LSTM)
    lrcn = LRCN(n_classes=n_classes, hidden_size=512, pretrained=pretrained, use_att=True).to(DEVICE)
    opt_lrcn = optim.Adam(lrcn.parameters(), lr=lr)
    best_val = 0.0
    best_model_path = f"{experiment_name}_lrcn_best.pth"
    print("Training LRCN...")
    for epoch in range(1, num_epochs+1):
        t0=time.time()
        loss = train_one_epoch(lrcn, train_loader, opt_lrcn, DEVICE, scaler=scaler)
        val_acc, _, _ = evaluate(lrcn, val_loader, DEVICE, label_names)
        print(f"[LRCN] Epoch {epoch}/{num_epochs} loss {loss:.4f} val_acc {val_acc:.4f} time {time.time()-t0:.1f}s")
        if val_acc > best_val:
            best_val = val_acc
            torch.save(lrcn.state_dict(), best_model_path)
    print("LRCN best val:", best_val)

    # 3) ConvLSTM
    convlstm = ConvLSTM(in_channels=3, hidden_channels=64, n_classes=n_classes).to(DEVICE)
    opt_convlstm = optim.Adam(convlstm.parameters(), lr=lr)
    best_val = 0.0
    best_model_path = f"{experiment_name}_convlstm_best.pth"
    print("Training ConvLSTM...")
    for epoch in range(1, num_epochs+1):
        t0=time.time()
        loss = train_one_epoch(convlstm, train_loader, opt_convlstm, DEVICE, scaler=scaler)
        val_acc, _, _ = evaluate(convlstm, val_loader, DEVICE, label_names)
        print(f"[ConvLSTM] Epoch {epoch}/{num_epochs} loss {loss:.4f} val_acc {val_acc:.4f} time {time.time()-t0:.1f}s")
        if val_acc > best_val:
            best_val = val_acc
            torch.save(convlstm.state_dict(), best_model_path)
    print("ConvLSTM best val:", best_val)

    # Evaluate saved best on test sets
    print("Evaluating best models on test set...")
    # Load & eval CNN
    cnn.load_state_dict(torch.load(f"{experiment_name}_cnn_best.pth"))
    cnn.to(DEVICE)
    acc_cnn, report_cnn, cm_cnn = evaluate(cnn, test_loader, DEVICE, label_names)
    print("CNN test acc:", acc_cnn)
    print(report_cnn)

    # LRCN
    lrcn.load_state_dict(torch.load(f"{experiment_name}_lrcn_best.pth"))
    lrcn.to(DEVICE)
    acc_lrcn, report_lrcn, cm_lrcn = evaluate(lrcn, test_loader, DEVICE, label_names)
    print("LRCN test acc:", acc_lrcn)
    print(report_lrcn)

    # ConvLSTM
    convlstm.load_state_dict(torch.load(f"{experiment_name}_convlstm_best.pth"))
    convlstm.to(DEVICE)
    acc_convlstm, report_convlstm, cm_convlstm = evaluate(convlstm, test_loader, DEVICE, label_names)
    print("ConvLSTM test acc:", acc_convlstm)
    print(report_convlstm)

    # Save confusion matrix images
    import seaborn as sns
    sns.set()
    def show_cm(cm, title):
        plt.figure(figsize=(6,5))
        sns.heatmap(cm, cmap='Blues', xticklabels=False, yticklabels=False)
        plt.title(title)
        plt.show()
    show_cm(cm_cnn, f"{experiment_name} CNN CM")
    show_cm(cm_lrcn, f"{experiment_name} LRCN CM")
    show_cm(cm_convlstm, f"{experiment_name} ConvLSTM CM")

    # Grad-CAM example using CNN backbone: take first test sample
    try:
        sample_frames, sample_lbl = test_ds[0]
        mid_frame = sample_frames[sample_frames.shape[0]//2]  # (C,H,W)
        # find a conv2d layer in CNN backbone
        target_layer = None
        for m in reversed(list(cnn.backbone.modules())):
            if isinstance(m, nn.Conv2d):
                target_layer = m; break
        if target_layer:
            gcam = GradCAM(cnn, target_layer)
            cam = gcam(mid_frame, None)
            img = mid_frame.permute(1,2,0).cpu().numpy()
            img = (img * np.array([0.229, 0.224, 0.225]) + np.array([0.485,0.456,0.406]))
            img = np.clip(img,0,1)
            heatmap = cv2.applyColorMap((cam*255).astype('uint8'), cv2.COLORMAP_JET)
            heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
            overlay = 0.4*heatmap + 0.6*img
            plt.figure(figsize=(6,6)); plt.imshow(overlay); plt.axis('off'); plt.title('Grad-CAM (sample)'); plt.show()
            gcam.close()
    except Exception as e:
        print("Grad-CAM example failed:", e)

    # Return summaries
    return {
        "cnn_test_acc": acc_cnn,
        "lrcn_test_acc": acc_lrcn,
        "convlstm_test_acc": acc_convlstm,
        "label_map": label_map
    }

In [ ]:

# -------------------------
# CELL 9 - Run two experiments (paper style): UCF50 and HMDB51
# Adjust parameters (epochs, batch size) for Colab GPU memory/time
# -------------------------
# Basic config
NUM_FRAMES = 16
IMAGE_SIZE = (224,224)
BATCH_SIZE = 8
NUM_EPOCHS = 30     # increase/decrease as you like
LR = 1e-4

# Ensure dataset roots exist
print("UCF50 root exists:", os.path.exists(UCF50_ROOT))
print("HMDB51 root exists:", os.path.exists(HMDB51_ROOT))

# Run UCF50 experiment (videos)
if os.path.exists(UCF50_ROOT):
    res_ucf = run_experiment(UCF50_ROOT, "UCF50", num_frames=NUM_FRAMES, image_size=IMAGE_SIZE,
                             batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, lr=LR, use_amp=True, pretrained=True)
else:
    print("UCF50 root missing. Skip UCF50 experiment")

# Run HMDB51 experiment (frames-per-video)
if os.path.exists(HMDB51_ROOT):
    res_hmdb = run_experiment(HMDB51_ROOT, "HMDB51", num_frames=NUM_FRAMES, image_size=IMAGE_SIZE,
                              batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS, lr=LR, use_amp=True, pretrained=True)
else:
    print("HMDB51 root missing. Skip HMDB51 experiment")

print("Done. Results (if any):")
try:
    print("UCF50:", res_ucf)
except:
    pass
try:
    print("HMDB51:", res_hmdb)
except:
    pass

UCF50 root exists: True
HMDB51 root exists: True
=== Experiment: UCF50 on root /content/drive/MyDrive/human-activity-recognition/data/UCF50/UCF50 ===
Counts (train/val/test): 4677 1002 1002
n_classes: 50


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:01<00:00, 58.6MB/s]
/tmp/ipython-input-325943398.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler() if (use_amp and DEVICE.type=='cuda') else None


Training CNN baseline...


...

Train:   0%|          | 0/585 [00:00<?, ?it/s]/tmp/ipython-input-321476418.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


[CNN] Epoch 9/30 loss 0.0759 val_acc 0.9152 time 554.8s


Train:   0%|          | 0/585 [00:00<?, ?it/s]/tmp/ipython-input-321476418.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


[CNN] Epoch 10/30 loss 0.0778 val_acc 0.9621 time 551.0s


Train:   0%|          | 0/585 [00:00<?, ?it/s]/tmp/ipython-input-321476418.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Train:  43%|████▎     | 252/585 [03:15<04:19,  1.28it/s]